In [ ]:
#loading and embedding documents (example)

from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="path/to/directory")
documents = reader.load_data()

In [2]:
from llama_index.core import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_overlap=0),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ]
)

nodes = await pipeline.arun(documents=[Document.example()])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\33781\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\33781\AppData\Local\llama_index\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
pip install llama-index-vector-stores-chroma

In [4]:
#storing and indexing documents
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=25, chunk_overlap=0),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)

In [10]:
# create an index from our vector store and embeddings
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

In [12]:
# Querying a VectorStoreIndex with prompts and LLMs

from llama_index.llms.ollama import Ollama

# Créer une instance du modèle local
llm = Ollama(
    model="qwen2:7b",  
    temperature=0.7,
    max_tokens=100,
)

"""
as_retriever:
    Utilisé pour la récupération basique de documents. 
    Retourne une liste d'objets NodeWithScore contenant les documents 
    les plus pertinents accompagnés de leur score de similarité.

as_query_engine:
    Adapté aux interactions de type question-réponse uniques. 
    Retourne une réponse textuelle générée à partir de la question 
    et du contexte indexé.

as_chat_engine:
    Conçu pour les interactions conversationnelles qui nécessitent une mémoire. 
    Maintient l'historique des échanges et utilise à la fois cet historique 
    et le contexte indexé pour produire une réponse cohérente.
"""


query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)

query_engine.query("What is the meaning of life?")

Response(response='Empty Response', source_nodes=[], metadata=None)

In [ ]:
#evaluation and observability

from llama_index.core.evaluation import FaithfulnessEvaluator


# query index
evaluator = FaithfulnessEvaluator(llm=llm)
response = query_engine.query(
    "What battles took place in New York City in the American Revolution?"
)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

False